## NY City Bike - Tableau Dashboard
----

### Notes
This notebook created to proccess and analyze the data sets
for the `NY Citi Bike Program` for 2019.

----

In [ ]:
# Import Libraries
import pandas as pd
import datetime
import math
import numpy as np

In [ ]:
# Function to calculate the distance between two points using coordinates
def distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 3956 # for miles

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d

### NY City Bike Dataset for 2019

In [ ]:
# Read file and print the number of records and data types
csv_name = "./2019/201901.csv"
jan_df = pd.read_csv(csv_name, low_memory=False) 
print("Records Jan-2019 : " + "\n" + str(jan_df.count()))

In [ ]:
# Add columns
jan_df.insert(0, 'year', 2019)
jan_df.insert(1, 'month', 'Jan')
jan_df.insert(3, 'tripdurmin', 0)
jan_df.insert(5, 'starthour', 0)
jan_df.insert(6, 'weekday', 0)
jan_df.insert(16, 'distance', 0)
jan_df.insert(21, 'age', 0)
jan_df.insert(22, 'sgender', '')
jan_df.insert(23, 'season', 'Winter')
jan_df.insert(24, 'mileage', 0)

jan_df.head(5)

In [ ]:
# Define values for gender
jan_df.loc[jan_df['gender'] == 0, 'sgender'] = 'Unknown'
jan_df.loc[jan_df['gender'] == 1, 'sgender'] = 'Male'
jan_df.loc[jan_df['gender'] == 2, 'sgender'] = 'Female'

In [ ]:
# Calculate the age of the person (in 2019)
jan_df['age'] = 2019 - jan_df['birth year']

In [ ]:
# Durantion of the trip from seconds to minutes
jan_df['tripdurmin'] = jan_df['tripduration'] / 60

In [ ]:
# Getting hour fron the 'startime'
jan_df['starthour'] = pd.to_datetime(jan_df['starttime']).dt.strftime('%H')

In [ ]:
# Getting weekday fron the 'startime'
jan_df['weekday'] = pd.to_datetime(jan_df['starttime']).dt.strftime('%A')

In [ ]:
# Calculate the distance in miles from start station to end station
jan_df['distance'] = jan_df.apply(lambda row: distance((row['start station latitude'],row['start station longitude']), 
                         (row['end station latitude'], row['end station longitude'])),
                         axis=1)

In [ ]:
# Transform the values for mileage estimates - assumed speed of 7.456 miles per hour, up to two hours. 
jan_df.loc[jan_df['tripdurmin'] <= 120, 'mileage'] = (jan_df['tripdurmin'] / 60) * 7.456

# Trips over 2 hrs max-out at 14.9 miles
jan_df.loc[jan_df['tripdurmin'] > 120, 'mileage'] = (jan_df['tripdurmin'] / 60) * 14.9

In [ ]:
# View results
jan_df.head()

### Data exploration (Jan-2019)

In [ ]:
# Data frame information
jan_df.info()

In [ ]:
# Count of customers and subscribers
jan_df.usertype.value_counts()

In [ ]:
# Peak hours for bike rentals
jan_df.starthour.value_counts()

In [ ]:
# Number of unique start stations
jan_df['start station id'].nunique()

In [ ]:
# Trips from all unique start stations
jan_df['start station id'].value_counts()

In [ ]:
# Number of unique end stations
jan_df['end station id'].nunique()

In [ ]:
# Arrivals to all unique end stations
jan_df['end station id'].value_counts()

In [ ]:
# Gender distribution of active participants.  Gender: Zero=unknown, 1=male, 2=female
jan_df.sgender.value_counts()

In [ ]:
# Average trip duration by age - Get the max and the min
# Min and Max Age = 2020 - birth year
print(f'Max age: {jan_df.age.max()}')
print(f'Min age: {jan_df.age.min()}')
print(f'Uniques: {jan_df.age.nunique()}')

In [ ]:
# Average trip duration by age - Differents values for age
# Age = 2020 - birth year 
jan_df.age.value_counts()

In [ ]:
# Number of trips by age
jan_df.groupby('age')['tripdurmin'].count()

In [ ]:
# Average trip duration by age
jan_df.groupby('age')['tripdurmin'].mean()

In [ ]:
# Average distance in miles for each unique bike
print(f'Uniques: {jan_df.bikeid.nunique()}')

In [ ]:
# Average distance in miles that a bike is ridden
jan_df.groupby(['bikeid']).agg({'distance': 'mean'}).sort_values(['distance'],ascending=False).head(5)

In [ ]:
# Which bikes (by ID) are most likely due for repair or inspection in the timespan?
jan_df.groupby(['bikeid']).agg({'distance': 'sum'}).sort_values(['distance'],ascending=False).head(5)

In [ ]:
# Variability by bike ID
jan_df.groupby(['bikeid']).agg({'distance': 'std'}).sort_values(['distance'],ascending=False).head(5)

### Assembling DataFrames for Tableau dashboard

#### --- January ---

In [ ]:
# Usertype + Gender
jan_user_df = jan_df.groupby(['usertype', 'sgender']).agg({i:'count' for i in jan_df.columns[19:20]})
jan_user_df.reset_index(inplace=True)
jan_user_df.rename(columns={"sgender":"gender", "birth year":"trips"}, inplace=True)
jan_user_df.insert(0, 'year', 2019)
jan_user_df.insert(1, 'month', 'Jan')
jan_user_df

In [ ]:
# Trips by hour, season and weekday
jan_season_df = jan_df.groupby(['starthour', 'season', 'weekday']).agg({i:'count' for i in jan_df.columns[19:20]})
jan_season_df.reset_index(inplace=True)
jan_season_df.rename(columns={"birth year":"trips"}, inplace=True)
jan_season_df.insert(0, 'year', 2019)
jan_season_df.insert(1, 'month', 'Jan')
jan_season_df

In [ ]:
# Start Stations
jan_stat_df = jan_df.groupby(['start station id', 'start station name', 'start station latitude', 'start station longitude']).agg({i:'count' for i in jan_df.columns[16:17]})
jan_stat_df.reset_index(inplace=True)
jan_stat_df.rename(columns={"start station id" : "stationid", "distance":"starttrips", "start station name": "startstatname"}, inplace=True)
jan_stat_df.insert(0, 'year', 2019)
jan_stat_df.insert(1, 'month', 'Jan')
jan_stat_df

In [ ]:
# End Stations
jan_statend_df = jan_df.groupby(['end station id', 'end station name', 'end station latitude', 'end station longitude']).agg({i:'count' for i in jan_df.columns[16:17]})
jan_statend_df.reset_index(inplace=True)
jan_statend_df.rename(columns={"end station id" : "stationid", "distance":"endtrips", "end station name": "endstatname"}, inplace=True)
jan_statend_df

In [ ]:
# Merge the start trips with the end trips by station id
jan_stat_df = jan_stat_df.merge(jan_statend_df, how="outer", left_on='stationid', right_on='stationid', suffixes=('_left', '_right'))
jan_stat_df

In [ ]:
# Ages vs Trip duration
jan_agedur_df = jan_df.groupby(by=['age','usertype'])['tripdurmin'].agg(['count', 'mean'])
jan_agedur_df.reset_index(inplace=True)
jan_agedur_df.insert(0, 'year', 2019)
jan_agedur_df.insert(1, 'month', 'Jan')
jan_agedur_df

In [ ]:
# Distance from start station to end station for each bike ID
jan_bike_df = jan_df.groupby(by=['bikeid'])['distance'].agg(['count', sum, 'mean', 'std'])
jan_bike_df.reset_index(inplace=True)
jan_bike_df.rename(columns={"count":"trips", "sum" : "totmiles", "mean" : "avgmiles", "std" : "stdmiles"}, inplace=True)
jan_bike_df.insert(0, 'year', 2019)
jan_bike_df.insert(1, 'month', 'Jan')
jan_bike_df

In [ ]:
# Calculate the total mileage for each bike ID
jan_bikemil_df = jan_df.groupby(by=['bikeid'])['mileage'].agg(['count', sum, 'mean', 'std'])
jan_bikemil_df.reset_index(inplace=True)
jan_bikemil_df.rename(columns={"count":"trips", "sum" : "totmileage", "mean" : "avgmileage", "std" : "stdmileage"}, inplace=True)
jan_bikemil_df.insert(0, 'year', 2019)
jan_bikemil_df.insert(1, 'month', 'Jan')
jan_bikemil_df

### Export DataFrames to CSV files (all remaining months data to be added in the next steps)

In [ ]:
jan_user_df.to_csv("./2019/user2019.csv", encoding="utf-8", index=False, header=True) 

jan_season_df.to_csv("./2019/season2019.csv", encoding="utf-8", index=False, header=True)

jan_stat_df.to_csv("./2019/station2019.csv", encoding="utf-8", index=False, header=True)

jan_agedur_df.to_csv("./2019/agedur2019.csv", encoding="utf-8", index=False, header=True)

jan_bike_df.to_csv("./2019/bike2019.csv", encoding="utf-8", index=False, header=True)

jan_bikemil_df.to_csv("./2019/mileage2019.csv", encoding="utf-8", index=False, header=True)

### Repeating same data processing for the remaining months of 2019

##### --- February ---

In [ ]:
# Read file and print the number of records and data types
csv_name = "./2019/201902.csv"
feb_df = pd.read_csv(csv_name, low_memory=False) 

In [ ]:
# Add columns
feb_df.insert(0, 'year', 2019)
feb_df.insert(1, 'month', 'Feb')
feb_df.insert(3, 'tripdurmin', 0)
feb_df.insert(5, 'starthour', 0)
feb_df.insert(6, 'weekday', 0)
feb_df.insert(16, 'distance', 0)
feb_df.insert(21, 'age', 0)
feb_df.insert(22, 'sgender', '')
feb_df.insert(23, 'season', 'Winter')
feb_df.insert(24, 'mileage', 0)
print("Records Feb-2019 : " + "\n" + str(feb_df.count()))

In [ ]:
# Define values for gender
feb_df.loc[feb_df['gender'] == 0, 'sgender'] = 'Unknown'
feb_df.loc[feb_df['gender'] == 1, 'sgender'] = 'Male'
feb_df.loc[feb_df['gender'] == 2, 'sgender'] = 'Female'

# Calculate the age of the person (in 2019)
feb_df['age'] = 2019 - feb_df['birth year']

# durantion of the trip from seconds to minutes
feb_df['tripdurmin'] = feb_df['tripduration'] / 60

# Getting hour fron the 'startime'
feb_df['starthour'] = pd.to_datetime(feb_df['starttime']).dt.strftime('%H')

# Getting weekday fron the 'startime'
feb_df['weekday'] = pd.to_datetime(feb_df['starttime']).dt.strftime('%A')

In [ ]:
# Calculate the distance in miles from start station to end station
feb_df['distance'] = feb_df.apply(lambda row: distance((row['start station latitude'],row['start station longitude']), 
                         (row['end station latitude'], row['end station longitude'])),
                         axis=1)

In [ ]:
# Transform the values for mileage estimates - assumed speed of 7.456 miles per hour, up to two hours. 
feb_df.loc[feb_df['tripdurmin'] <= 120, 'mileage'] = (feb_df['tripdurmin'] / 60) * 7.456

# Trips over 2 hrs max-out at 14.9 miles
feb_df.loc[feb_df['tripdurmin'] > 120, 'mileage'] = (feb_df['tripdurmin'] / 60) * 14.9

In [ ]:
# Usertype + Gender
feb_user_df =feb_df.groupby(['usertype', 'sgender']).agg({i:'count' for i in feb_df.columns[19:20]})
feb_user_df.reset_index(inplace=True)
feb_user_df.rename(columns={"sgender":"gender", "birth year":"trips"}, inplace=True)
feb_user_df.insert(0, 'year', 2019)
feb_user_df.insert(1, 'month', 'Feb')
feb_user_df

In [ ]:
# Trips by hour, season and weekday
feb_season_df = feb_df.groupby(['starthour', 'season', 'weekday']).agg({i:'count' for i in feb_df.columns[19:20]})
feb_season_df.reset_index(inplace=True)
feb_season_df.rename(columns={"birth year":"trips"}, inplace=True)
feb_season_df.insert(0, 'year', 2019)
feb_season_df.insert(1, 'month', 'Feb')
feb_season_df

In [ ]:
# Start Stations
feb_stat_df = feb_df.groupby(['start station id', 'start station name', 'start station latitude', 'start station longitude']).agg({i:'count' for i in feb_df.columns[16:17]})
feb_stat_df.reset_index(inplace=True)
feb_stat_df.rename(columns={"start station id" : "stationid", "distance":"starttrips", "start station name": "startstatname"}, inplace=True)
feb_stat_df.insert(0, 'year', 2019)
feb_stat_df.insert(1, 'month', 'Feb')
feb_stat_df

In [ ]:
# End Stations
feb_statend_df = feb_df.groupby(['end station id', 'end station name', 'end station latitude', 'end station longitude']).agg({i:'count' for i in feb_df.columns[16:17]})
feb_statend_df.reset_index(inplace=True)
feb_statend_df.rename(columns={"end station id" : "stationid", "distance":"endtrips", "end station name": "endstatname"}, inplace=True)
feb_statend_df

In [ ]:
# Merge the start trips with the end trips by station id
feb_stat_df = feb_stat_df.merge(feb_statend_df, how="outer", left_on='stationid', right_on='stationid', suffixes=('_left', '_right'))
feb_stat_df

In [ ]:
# Ages vs Trip duration
feb_agedur_df = feb_df.groupby(by=['age','usertype'])['tripdurmin'].agg(['count', 'mean'])
feb_agedur_df.reset_index(inplace=True)
feb_agedur_df.insert(0, 'year', 2019)
feb_agedur_df.insert(1, 'month', 'Feb')
feb_agedur_df

In [ ]:
# Distance from start station to end station for each bike ID
feb_bike_df = feb_df.groupby(by=['bikeid'])['distance'].agg(['count', sum, 'mean', 'std'])
feb_bike_df.reset_index(inplace=True)
feb_bike_df.rename(columns={"count":"trips", "sum" : "totmiles", "mean" : "avgmiles", "std" : "stdmiles"}, inplace=True)
feb_bike_df.insert(0, 'year', 2019)
feb_bike_df.insert(1, 'month', 'Feb')
feb_bike_df

In [ ]:
# Calculate the total mileage for each bike ID
feb_bikemil_df = feb_df.groupby(by=['bikeid'])['mileage'].agg(['count', sum, 'mean', 'std'])
feb_bikemil_df.reset_index(inplace=True)
feb_bikemil_df.rename(columns={"count":"trips", "sum" : "totmileage", "mean" : "avgmileage", "std" : "stdmileage"}, inplace=True)
feb_bikemil_df.insert(0, 'year', 2019)
feb_bikemil_df.insert(1, 'month', 'Feb')
feb_bikemil_df

In [ ]:
# Append DataFrames to CSVs
with open('./2019/user2019.csv', 'a') as f:
    feb_user_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/season2019.csv', 'a') as f:
    feb_season_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/station2019.csv', 'a') as f:
    feb_stat_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/agedur2019.csv', 'a') as f:
    feb_agedur_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/bike2019.csv', 'a') as f:
    feb_bike_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/mileage2019.csv', 'a') as f:
    feb_bikemil_df.to_csv(f, encoding="utf-8", index=False, header=False)

##### --- March ---

In [ ]:
# Read file and print the number of records and data types
csv_name = "./2019/201903.csv"
mar_df = pd.read_csv(csv_name, low_memory=False) 

In [ ]:
# Add columns
mar_df.insert(0, 'year', 2019)
mar_df.insert(1, 'month', 'Mar')
mar_df.insert(3, 'tripdurmin', 0)
mar_df.insert(5, 'starthour', 0)
mar_df.insert(6, 'weekday', 0)
mar_df.insert(16, 'distance', 0)
mar_df.insert(21, 'age', 0)
mar_df.insert(22, 'sgender', '')
mar_df.insert(23, 'season', 'Spring')
mar_df.insert(24, 'mileage', 0)
print("Records Mar-2019 : " + "\n" + str(mar_df.count()))

In [ ]:
# Define values for gender
mar_df.loc[mar_df['gender'] == 0, 'sgender'] = 'Unknown'
mar_df.loc[mar_df['gender'] == 1, 'sgender'] = 'Male'
mar_df.loc[mar_df['gender'] == 2, 'sgender'] = 'Female'

# Calculate the age of the person (in 2019)
mar_df['age'] = 2019 - mar_df['birth year']

# durantion of the trip from seconds to minutes
mar_df['tripdurmin'] = mar_df['tripduration'] / 60

# Getting hour fron the 'startime'
mar_df['starthour'] = pd.to_datetime(mar_df['starttime']).dt.strftime('%H')

# Getting weekday fron the 'startime'
mar_df['weekday'] = pd.to_datetime(mar_df['starttime']).dt.strftime('%A')

In [ ]:
# Calculate the distance in miles from start station to end station
mar_df['distance'] = mar_df.apply(lambda row: distance((row['start station latitude'],row['start station longitude']), 
                         (row['end station latitude'], row['end station longitude'])),
                         axis=1)

In [ ]:
# Transform the values for mileage estimates - assumed speed of 7.456 miles per hour, up to two hours. 
mar_df.loc[mar_df['tripdurmin'] <= 120, 'mileage'] = (mar_df['tripdurmin'] / 60) * 7.456

# Trips over 2 hrs max-out at 14.9 miles
mar_df.loc[mar_df['tripdurmin'] > 120, 'mileage'] = (mar_df['tripdurmin'] / 60) * 14.9

In [ ]:
# Usertype + Gender
mar_user_df =mar_df.groupby(['usertype', 'sgender']).agg({i:'count' for i in mar_df.columns[19:20]})
mar_user_df.reset_index(inplace=True)
mar_user_df.rename(columns={"sgender":"gender", "birth year":"trips"}, inplace=True)
mar_user_df.insert(0, 'year', 2019)
mar_user_df.insert(1, 'month', 'Mar')
mar_user_df

In [ ]:
# Trips by hour, season and weekday
mar_season_df = mar_df.groupby(['starthour', 'season', 'weekday']).agg({i:'count' for i in mar_df.columns[19:20]})
mar_season_df.reset_index(inplace=True)
mar_season_df.rename(columns={"birth year":"trips"}, inplace=True)
mar_season_df.insert(0, 'year', 2019)
mar_season_df.insert(1, 'month', 'Mar')
mar_season_df

In [ ]:
# Start Stations
mar_stat_df = mar_df.groupby(['start station id', 'start station name', 'start station latitude', 'start station longitude']).agg({i:'count' for i in mar_df.columns[16:17]})
mar_stat_df.reset_index(inplace=True)
mar_stat_df.rename(columns={"start station id" : "stationid", "distance":"starttrips", "start station name": "startstatname"}, inplace=True)
mar_stat_df.insert(0, 'year', 2019)
mar_stat_df.insert(1, 'month', 'Mar')
mar_stat_df

In [ ]:
# End Stations
mar_statend_df = mar_df.groupby(['end station id', 'end station name', 'end station latitude', 'end station longitude']).agg({i:'count' for i in mar_df.columns[16:17]})
mar_statend_df.reset_index(inplace=True)
mar_statend_df.rename(columns={"end station id" : "stationid", "distance":"endtrips", "end station name": "endstatname"}, inplace=True)
mar_statend_df

In [ ]:
# Merge the start trips with the end trips by station id
mar_stat_df = mar_stat_df.merge(mar_statend_df, how="outer", left_on='stationid', right_on='stationid', suffixes=('_left', '_right'))
mar_stat_df

In [ ]:
# Ages vs Trip duration
mar_agedur_df = mar_df.groupby(by=['age','usertype'])['tripdurmin'].agg(['count', 'mean'])
mar_agedur_df.reset_index(inplace=True)
mar_agedur_df.insert(0, 'year', 2019)
mar_agedur_df.insert(1, 'month', 'Mar')
mar_agedur_df

In [ ]:
# Distance from start station to end station for each bike ID
mar_bike_df = mar_df.groupby(by=['bikeid'])['distance'].agg(['count', sum, 'mean', 'std'])
mar_bike_df.reset_index(inplace=True)
mar_bike_df.rename(columns={"count":"trips", "sum" : "totmiles", "mean" : "avgmiles", "std" : "stdmiles"}, inplace=True)
mar_bike_df.insert(0, 'year', 2020)
mar_bike_df.insert(1, 'month', 'Mar')
mar_bike_df

In [ ]:
# Calculate the total mileage for each bike ID
mar_bikemil_df = mar_df.groupby(by=['bikeid'])['mileage'].agg(['count', sum, 'mean', 'std'])
mar_bikemil_df.reset_index(inplace=True)
mar_bikemil_df.rename(columns={"count":"trips", "sum" : "totmileage", "mean" : "avgmileage", "std" : "stdmileage"}, inplace=True)
mar_bikemil_df.insert(0, 'year', 2019)
mar_bikemil_df.insert(1, 'month', 'Mar')
mar_bikemil_df

In [ ]:
# Append DataFrames to CSVs
with open('./2019/user2019.csv', 'a') as f:
    mar_user_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/season2019.csv', 'a') as f:
    mar_season_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/station2019.csv', 'a') as f:
    mar_stat_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/agedur2019.csv', 'a') as f:
    mar_agedur_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/bike2019.csv', 'a') as f:
    mar_bike_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/mileage2019.csv', 'a') as f:
    mar_bikemil_df.to_csv(f, encoding="utf-8", index=False, header=False)

##### --- April ---

In [ ]:
# Read file and print the number of records and data types
csv_name = "./2019/201904.csv"
apr_df = pd.read_csv(csv_name, low_memory=False) 

In [ ]:
# Add columns
apr_df.insert(0, 'year', 2019)
apr_df.insert(1, 'month', 'Apr')
apr_df.insert(3, 'tripdurmin', 0)
apr_df.insert(5, 'starthour', 0)
apr_df.insert(6, 'weekday', 0)
apr_df.insert(16, 'distance', 0)
apr_df.insert(21, 'age', 0)
apr_df.insert(22, 'sgender', '')
apr_df.insert(23, 'season', 'Spring')
apr_df.insert(24, 'mileage', 0)
print("Records Apr-2019 : " + "\n" + str(apr_df.count()))

In [ ]:
# Define values for gender
apr_df.loc[apr_df['gender'] == 0, 'sgender'] = 'Unknown'
apr_df.loc[apr_df['gender'] == 1, 'sgender'] = 'Male'
apr_df.loc[apr_df['gender'] == 2, 'sgender'] = 'Female'

# Calculate the age of the person (in 2019)
apr_df['age'] = 2019 - apr_df['birth year']

# durantion of the trip from seconds to minutes
apr_df['tripdurmin'] = apr_df['tripduration'] / 60

# Getting hour fron the 'startime'
apr_df['starthour'] = pd.to_datetime(apr_df['starttime']).dt.strftime('%H')

# Getting weekday fron the 'startime'
apr_df['weekday'] = pd.to_datetime(apr_df['starttime']).dt.strftime('%A')

In [ ]:
# Calculate the distance in miles from start station to end station
apr_df['distance'] = apr_df.apply(lambda row: distance((row['start station latitude'],row['start station longitude']), 
                         (row['end station latitude'], row['end station longitude'])),
                         axis=1)

In [ ]:
# Transform the values for mileage estimates - assumed speed of 7.456 miles per hour, up to two hours. 
apr_df.loc[apr_df['tripdurmin'] <= 120, 'mileage'] = (apr_df['tripdurmin'] / 60) * 7.456

# Trips over 2 hrs max-out at 14.9 miles
apr_df.loc[apr_df['tripdurmin'] > 120, 'mileage'] = (apr_df['tripdurmin'] / 60) * 14.9

In [ ]:
# Usertype + Gender
apr_user_df =apr_df.groupby(['usertype', 'sgender']).agg({i:'count' for i in apr_df.columns[19:20]})
apr_user_df.reset_index(inplace=True)
apr_user_df.rename(columns={"sgender":"gender", "birth year":"trips"}, inplace=True)
apr_user_df.insert(0, 'year', 2019)
apr_user_df.insert(1, 'month', 'Apr')
apr_user_df

In [ ]:
# Trips by hour, season and weekday
apr_season_df = apr_df.groupby(['starthour', 'season', 'weekday']).agg({i:'count' for i in apr_df.columns[19:20]})
apr_season_df.reset_index(inplace=True)
apr_season_df.rename(columns={"birth year":"trips"}, inplace=True)
apr_season_df.insert(0, 'year', 2019)
apr_season_df.insert(1, 'month', 'Apr')
apr_season_df

In [ ]:
# Start Stations
apr_stat_df = apr_df.groupby(['start station id', 'start station name', 'start station latitude', 'start station longitude']).agg({i:'count' for i in apr_df.columns[16:17]})
apr_stat_df.reset_index(inplace=True)
apr_stat_df.rename(columns={"start station id" : "stationid", "distance":"starttrips", "start station name": "startstatname"}, inplace=True)
apr_stat_df.insert(0, 'year', 2020)
apr_stat_df.insert(1, 'month', 'Apr')
apr_stat_df

In [ ]:
# End Stations
apr_statend_df = apr_df.groupby(['end station id', 'end station name', 'end station latitude', 'end station longitude']).agg({i:'count' for i in apr_df.columns[16:17]})
apr_statend_df.reset_index(inplace=True)
apr_statend_df.rename(columns={"end station id" : "stationid", "distance":"endtrips", "end station name": "endstatname"}, inplace=True)
apr_statend_df

In [ ]:
# Merge the start trips with the end trips by station id
apr_stat_df = apr_stat_df.merge(apr_statend_df, how="outer", left_on='stationid', right_on='stationid', suffixes=('_left', '_right'))
apr_stat_df

In [ ]:
# Ages vs Trip duration
apr_agedur_df = apr_df.groupby(by=['age','usertype'])['tripdurmin'].agg(['count', 'mean'])
apr_agedur_df.reset_index(inplace=True)
apr_agedur_df.insert(0, 'year', 2019)
apr_agedur_df.insert(1, 'month', 'Apr')
apr_agedur_df

In [ ]:
# Distance from start station to end station for each bike ID
apr_bike_df = apr_df.groupby(by=['bikeid'])['distance'].agg(['count', sum, 'mean', 'std'])
apr_bike_df.reset_index(inplace=True)
apr_bike_df.rename(columns={"count":"trips", "sum" : "totmiles", "mean" : "avgmiles", "std" : "stdmiles"}, inplace=True)
apr_bike_df.insert(0, 'year', 2019)
apr_bike_df.insert(1, 'month', 'Apr')
apr_bike_df

In [ ]:
# Calculate the total mileage for each bike ID
apr_bikemil_df = apr_df.groupby(by=['bikeid'])['mileage'].agg(['count', sum, 'mean', 'std'])
apr_bikemil_df.reset_index(inplace=True)
apr_bikemil_df.rename(columns={"count":"trips", "sum" : "totmileage", "mean" : "avgmileage", "std" : "stdmileage"}, inplace=True)
apr_bikemil_df.insert(0, 'year', 2019)
apr_bikemil_df.insert(1, 'month', 'Apr')
apr_bikemil_df

In [ ]:
# Append DataFrames to CSVs
with open('./2019/user2019.csv', 'a') as f:
    apr_user_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/season2019.csv', 'a') as f:
    apr_season_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/station2019.csv', 'a') as f:
    apr_stat_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/agedur2019.csv', 'a') as f:
    apr_agedur_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2020/bike2020.csv', 'a') as f:
    apr_bike_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2020/mileage2020.csv', 'a') as f:
    apr_bikemil_df.to_csv(f, encoding="utf-8", index=False, header=False)

##### --- May ---

In [ ]:
# Read file and print the number of records and data types
csv_name = "./2019/201905.csv"
may_df = pd.read_csv(csv_name, low_memory=False) 

In [ ]:
# Add columns
may_df.insert(0, 'year', 2019)
may_df.insert(1, 'month', 'May')
may_df.insert(3, 'tripdurmin', 0)
may_df.insert(5, 'starthour', 0)
may_df.insert(6, 'weekday', 0)
may_df.insert(16, 'distance', 0)
may_df.insert(21, 'age', 0)
may_df.insert(22, 'sgender', '')
may_df.insert(23, 'season', 'Spring')
may_df.insert(24, 'mileage', 0)
print("Records May-2019 : " + "\n" + str(may_df.count()))

In [ ]:
# Define values for gender
may_df.loc[may_df['gender'] == 0, 'sgender'] = 'Unknown'
may_df.loc[may_df['gender'] == 1, 'sgender'] = 'Male'
may_df.loc[may_df['gender'] == 2, 'sgender'] = 'Female'

# Calculate the age of the person (in 2019)
may_df['age'] = 2019 - may_df['birth year']

# durantion of the trip from seconds to minutes
may_df['tripdurmin'] = may_df['tripduration'] / 60

# Getting hour fron the 'startime'
may_df['starthour'] = pd.to_datetime(may_df['starttime']).dt.strftime('%H')

# Getting weekday fron the 'startime'
may_df['weekday'] = pd.to_datetime(may_df['starttime']).dt.strftime('%A')

In [ ]:
# Calculate the distance in miles from start station to end station
may_df['distance'] = may_df.apply(lambda row: distance((row['start station latitude'],row['start station longitude']), 
                         (row['end station latitude'], row['end station longitude'])),
                         axis=1)

In [ ]:
# Transform the values for mileage estimates - assumed speed of 7.456 miles per hour, up to two hours. 
may_df.loc[may_df['tripdurmin'] <= 120, 'mileage'] = (may_df['tripdurmin'] / 60) * 7.456

# Trips over 2 hrs max-out at 14.9 miles
may_df.loc[may_df['tripdurmin'] > 120, 'mileage'] = (may_df['tripdurmin'] / 60) * 14.9

In [ ]:
# Usertype + Gender
may_user_df =may_df.groupby(['usertype', 'sgender']).agg({i:'count' for i in may_df.columns[19:20]})
may_user_df.reset_index(inplace=True)
may_user_df.rename(columns={"sgender":"gender", "birth year":"trips"}, inplace=True)
may_user_df.insert(0, 'year', 2019)
may_user_df.insert(1, 'month', 'May')
may_user_df

In [ ]:
# Trips by hour, season and weekday
may_season_df = may_df.groupby(['starthour', 'season', 'weekday']).agg({i:'count' for i in may_df.columns[19:20]})
may_season_df.reset_index(inplace=True)
may_season_df.rename(columns={"birth year":"trips"}, inplace=True)
may_season_df.insert(0, 'year', 2019)
may_season_df.insert(1, 'month', 'May')
may_season_df

In [ ]:
# Start Stations
may_stat_df = may_df.groupby(['start station id', 'start station name', 'start station latitude', 'start station longitude']).agg({i:'count' for i in may_df.columns[16:17]})
may_stat_df.reset_index(inplace=True)
may_stat_df.rename(columns={"start station id" : "stationid", "distance":"starttrips", "start station name": "startstatname"}, inplace=True)
may_stat_df.insert(0, 'year', 2019)
may_stat_df.insert(1, 'month', 'May')
may_stat_df

In [ ]:
# End Stations
may_statend_df = may_df.groupby(['end station id', 'end station name', 'end station latitude', 'end station longitude']).agg({i:'count' for i in may_df.columns[16:17]})
may_statend_df.reset_index(inplace=True)
may_statend_df.rename(columns={"end station id" : "stationid", "distance":"endtrips", "end station name": "endstatname"}, inplace=True)
may_statend_df

In [ ]:
# Merge the start trips with the end trips by station id
may_stat_df = may_stat_df.merge(may_statend_df, how="outer", left_on='stationid', right_on='stationid', suffixes=('_left', '_right'))
may_stat_df

In [ ]:
# Ages vs Trip duration
may_agedur_df = may_df.groupby(by=['age','usertype'])['tripdurmin'].agg(['count', 'mean'])
may_agedur_df.reset_index(inplace=True)
may_agedur_df.insert(0, 'year', 2019)
may_agedur_df.insert(1, 'month', 'May')
may_agedur_df

In [ ]:
# Distance from start station to end station for each bike ID
may_bike_df = may_df.groupby(by=['bikeid'])['distance'].agg(['count', sum, 'mean', 'std'])
may_bike_df.reset_index(inplace=True)
may_bike_df.rename(columns={"count":"trips", "sum" : "totmiles", "mean" : "avgmiles", "std" : "stdmiles"}, inplace=True)
may_bike_df.insert(0, 'year', 2019)
may_bike_df.insert(1, 'month', 'May')
may_bike_df

In [ ]:
# Calculate the total mileage for each bike ID
may_bikemil_df = may_df.groupby(by=['bikeid'])['mileage'].agg(['count', sum, 'mean', 'std'])
may_bikemil_df.reset_index(inplace=True)
may_bikemil_df.rename(columns={"count":"trips", "sum" : "totmileage", "mean" : "avgmileage", "std" : "stdmileage"}, inplace=True)
may_bikemil_df.insert(0, 'year', 2019)
may_bikemil_df.insert(1, 'month', 'May')
may_bikemil_df

In [ ]:
# Append DataFrames to CSVs
with open('./2019/user2019.csv', 'a') as f:
    may_user_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/season2019.csv', 'a') as f:
    may_season_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/station2019.csv', 'a') as f:
    may_stat_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/agedur2019.csv', 'a') as f:
    may_agedur_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/bike2019.csv', 'a') as f:
    may_bike_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/mileage2019.csv', 'a') as f:
    may_bikemil_df.to_csv(f, encoding="utf-8", index=False, header=False)

##### --- Jun ---

In [ ]:
# Read file and print the number of records and data types
csv_name = "./2019/201906.csv"
jun_df = pd.read_csv(csv_name, low_memory=False) 

In [ ]:
# Add columns
jun_df.insert(0, 'year', 2019)
jun_df.insert(1, 'month', 'Jun')
jun_df.insert(3, 'tripdurmin', 0)
jun_df.insert(5, 'starthour', 0)
jun_df.insert(6, 'weekday', 0)
jun_df.insert(16, 'distance', 0)
jun_df.insert(21, 'age', 0)
jun_df.insert(22, 'sgender', '')
jun_df.insert(23, 'season', 'Summer')
jun_df.insert(24, 'mileage', 0)
print("Records Jun-2019 : " + "\n" + str(jun_df.count()))

In [ ]:
# Define values for gender
jun_df.loc[jun_df['gender'] == 0, 'sgender'] = 'Unknown'
jun_df.loc[jun_df['gender'] == 1, 'sgender'] = 'Male'
jun_df.loc[jun_df['gender'] == 2, 'sgender'] = 'Female'

# Calculate the age of the person (in 2019)
jun_df['age'] = 2019 - jun_df['birth year']

# durantion of the trip from seconds to minutes
jun_df['tripdurmin'] = jun_df['tripduration'] / 60

# Getting hour fron the 'startime'
jun_df['starthour'] = pd.to_datetime(jun_df['starttime']).dt.strftime('%H')

# Getting weekday fron the 'startime'
jun_df['weekday'] = pd.to_datetime(jun_df['starttime']).dt.strftime('%A')

In [ ]:
# Calculate the distance in miles from start station to end station
jun_df['distance'] = jun_df.apply(lambda row: distance((row['start station latitude'],row['start station longitude']), 
                         (row['end station latitude'], row['end station longitude'])),
                         axis=1)

In [ ]:
# Transform the values for mileage estimates - assumed speed of 7.456 miles per hour, up to two hours. 
jun_df.loc[jun_df['tripdurmin'] <= 120, 'mileage'] = (jun_df['tripdurmin'] / 60) * 7.456

# Trips over 2 hrs max-out at 14.9 miles
jun_df.loc[jun_df['tripdurmin'] > 120, 'mileage'] = (jun_df['tripdurmin'] / 60) * 14.9

In [ ]:
# Usertype + Gender
jun_user_df = jun_df.groupby(['usertype', 'sgender']).agg({i:'count' for i in jun_df.columns[19:20]})
jun_user_df.reset_index(inplace=True)
jun_user_df.rename(columns={"sgender":"gender", "birth year":"trips"}, inplace=True)
jun_user_df.insert(0, 'year', 2019)
jun_user_df.insert(1, 'month', 'Jun')
jun_user_df

In [ ]:
# Trips by hour, season and weekday
jun_season_df = jun_df.groupby(['starthour', 'season', 'weekday']).agg({i:'count' for i in jun_df.columns[19:20]})
jun_season_df.reset_index(inplace=True)
jun_season_df.rename(columns={"birth year":"trips"}, inplace=True)
jun_season_df.insert(0, 'year', 2019)
jun_season_df.insert(1, 'month', 'Jun')
jun_season_df

In [ ]:
# Start Stations
jun_stat_df = jun_df.groupby(['start station id', 'start station name', 'start station latitude', 'start station longitude']).agg({i:'count' for i in jun_df.columns[16:17]})
jun_stat_df.reset_index(inplace=True)
jun_stat_df.rename(columns={"start station id" : "stationid", "distance":"starttrips", "start station name": "startstatname"}, inplace=True)
jun_stat_df.insert(0, 'year', 2019)
jun_stat_df.insert(1, 'month', 'Jun')
jun_stat_df

In [ ]:
# End Stations
jun_statend_df = may_df.groupby(['end station id', 'end station name', 'end station latitude', 'end station longitude']).agg({i:'count' for i in may_df.columns[16:17]})
jun_statend_df.reset_index(inplace=True)
jun_statend_df.rename(columns={"end station id" : "stationid", "distance":"endtrips", "end station name": "endstatname"}, inplace=True)
jun_statend_df

In [ ]:
# Merge the start trips with the end trips by station id
jun_stat_df = jun_stat_df.merge(jun_statend_df, how="outer", left_on='stationid', right_on='stationid', suffixes=('_left', '_right'))
jun_stat_df

In [ ]:
# Ages vs Trip duration
jun_agedur_df = jun_df.groupby(by=['age','usertype'])['tripdurmin'].agg(['count', 'mean'])
jun_agedur_df.reset_index(inplace=True)
jun_agedur_df.insert(0, 'year', 2019)
jun_agedur_df.insert(1, 'month', 'Jun')
jun_agedur_df

In [ ]:
# Distance from start station to end station for each bike ID
jun_bike_df = jun_df.groupby(by=['bikeid'])['distance'].agg(['count', sum, 'mean', 'std'])
jun_bike_df.reset_index(inplace=True)
jun_bike_df.rename(columns={"count":"trips", "sum" : "totmiles", "mean" : "avgmiles", "std" : "stdmiles"}, inplace=True)
jun_bike_df.insert(0, 'year', 2019)
jun_bike_df.insert(1, 'month', 'Jun')
jun_bike_df

In [ ]:
# Calculate the total mileage for each bike ID
jun_bikemil_df = jun_df.groupby(by=['bikeid'])['mileage'].agg(['count', sum, 'mean', 'std'])
jun_bikemil_df.reset_index(inplace=True)
jun_bikemil_df.rename(columns={"count":"trips", "sum" : "totmileage", "mean" : "avgmileage", "std" : "stdmileage"}, inplace=True)
jun_bikemil_df.insert(0, 'year', 2019)
jun_bikemil_df.insert(1, 'month', 'Jun')
jun_bikemil_df

In [ ]:
# Append DataFrames to CSVs
with open('./2019/user2019.csv', 'a') as f:
    jun_user_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/season2019.csv', 'a') as f:
    jun_season_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/station2019.csv', 'a') as f:
    jun_stat_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/agedur2019.csv', 'a') as f:
    jun_agedur_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/bike2019.csv', 'a') as f:
    jun_bike_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/mileage2019.csv', 'a') as f:
    jun_bikemil_df.to_csv(f, encoding="utf-8", index=False, header=False)

##### --- July ---

In [ ]:
# Read file and print the number of records and data types
csv_name = "./2019/201907.csv"
jul_df = pd.read_csv(csv_name, low_memory=False) 

In [ ]:
# Add columns
jul_df.insert(0, 'year', 2019)
jul_df.insert(1, 'month', 'Jul')
jul_df.insert(3, 'tripdurmin', 0)
jul_df.insert(5, 'starthour', 0)
jul_df.insert(6, 'weekday', 0)
jul_df.insert(16, 'distance', 0)
jul_df.insert(21, 'age', 0)
jul_df.insert(22, 'sgender', '')
jul_df.insert(23, 'season', 'Summer')
jul_df.insert(24, 'mileage', 0)
print("Records Jul-2019 : " + "\n" + str(jul_df.count()))

In [ ]:
# Define values for gender
jul_df.loc[jul_df['gender'] == 0, 'sgender'] = 'Unknown'
jul_df.loc[jul_df['gender'] == 1, 'sgender'] = 'Male'
jul_df.loc[jul_df['gender'] == 2, 'sgender'] = 'Female'

# Calculate the age of the person (in 2019)
jul_df['age'] = 2019 - jul_df['birth year']

# durantion of the trip from seconds to minutes
jul_df['tripdurmin'] = jul_df['tripduration'] / 60

# Getting hour fron the 'startime'
jul_df['starthour'] = pd.to_datetime(jul_df['starttime']).dt.strftime('%H')

# Getting weekday fron the 'startime'
jul_df['weekday'] = pd.to_datetime(jul_df['starttime']).dt.strftime('%A')

In [ ]:
# Calculate the distance in miles from start station to end station
jul_df['distance'] = jul_df.apply(lambda row: distance((row['start station latitude'],row['start station longitude']), 
                         (row['end station latitude'], row['end station longitude'])),
                         axis=1)

In [ ]:
# Transform the values for mileage estimates - assumed speed of 7.456 miles per hour, up to two hours. 
jul_df.loc[jul_df['tripdurmin'] <= 120, 'mileage'] = (jul_df['tripdurmin'] / 60) * 7.456

# Trips over 2 hrs max-out at 14.9 miles
jul_df.loc[jul_df['tripdurmin'] > 120, 'mileage'] = (jul_df['tripdurmin'] / 60) * 14.9

In [ ]:
# Usertype + Gender
jul_user_df = jul_df.groupby(['usertype', 'sgender']).agg({i:'count' for i in jul_df.columns[19:20]})
jul_user_df.reset_index(inplace=True)
jul_user_df.rename(columns={"sgender":"gender", "birth year":"trips"}, inplace=True)
jul_user_df.insert(0, 'year', 2019)
jul_user_df.insert(1, 'month', 'Jul')
jul_user_df

In [ ]:
# Trips by hour, season and weekday
jul_season_df = jul_df.groupby(['starthour', 'season', 'weekday']).agg({i:'count' for i in jul_df.columns[19:20]})
jul_season_df.reset_index(inplace=True)
jul_season_df.rename(columns={"birth year":"trips"}, inplace=True)
jul_season_df.insert(0, 'year', 2019)
jul_season_df.insert(1, 'month', 'Jul')
jul_season_df

In [ ]:
# Start Stations
jul_stat_df = jul_df.groupby(['start station id', 'start station name', 'start station latitude', 'start station longitude']).agg({i:'count' for i in jul_df.columns[16:17]})
jul_stat_df.reset_index(inplace=True)
jul_stat_df.rename(columns={"start station id" : "stationid", "distance":"starttrips", "start station name": "startstatname"}, inplace=True)
jul_stat_df.insert(0, 'year', 2019)
jul_stat_df.insert(1, 'month', 'Jul')
jul_stat_df

In [ ]:
# End Stations
jul_statend_df = may_df.groupby(['end station id', 'end station name', 'end station latitude', 'end station longitude']).agg({i:'count' for i in may_df.columns[16:17]})
jul_statend_df.reset_index(inplace=True)
jul_statend_df.rename(columns={"end station id" : "stationid", "distance":"endtrips", "end station name": "endstatname"}, inplace=True)
jul_statend_df

In [ ]:
# Merge the start trips with the end trips by station id
jul_stat_df = jul_stat_df.merge(jul_statend_df, how="outer", left_on='stationid', right_on='stationid', suffixes=('_left', '_right'))
jul_stat_df

In [ ]:
# Ages vs Trip duration
jul_agedur_df = jul_df.groupby(by=['age','usertype'])['tripdurmin'].agg(['count', 'mean'])
jul_agedur_df.reset_index(inplace=True)
jul_agedur_df.insert(0, 'year', 2019)
jul_agedur_df.insert(1, 'month', 'Jul')
jul_agedur_df

In [ ]:
# Distance from start station to end station for each bike ID
jul_bike_df = jul_df.groupby(by=['bikeid'])['distance'].agg(['count', sum, 'mean', 'std'])
jul_bike_df.reset_index(inplace=True)
jul_bike_df.rename(columns={"count":"trips", "sum" : "totmiles", "mean" : "avgmiles", "std" : "stdmiles"}, inplace=True)
jul_bike_df.insert(0, 'year', 2019)
jul_bike_df.insert(1, 'month', 'Jul')
jul_bike_df

In [ ]:
# Calculate the total mileage for each bike ID
jul_bikemil_df = jul_df.groupby(by=['bikeid'])['mileage'].agg(['count', sum, 'mean', 'std'])
jul_bikemil_df.reset_index(inplace=True)
jul_bikemil_df.rename(columns={"count":"trips", "sum" : "totmileage", "mean" : "avgmileage", "std" : "stdmileage"}, inplace=True)
jul_bikemil_df.insert(0, 'year', 2019)
jul_bikemil_df.insert(1, 'month', 'Jul')
jul_bikemil_df

In [ ]:
# Append DataFrames to CSVs
with open('./2019/user2019.csv', 'a') as f:
    jul_user_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/season2019.csv', 'a') as f:
    jul_season_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/station2019.csv', 'a') as f:
    jul_stat_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/agedur2019.csv', 'a') as f:
    jul_agedur_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/bike2019.csv', 'a') as f:
    jul_bike_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/mileage2019.csv', 'a') as f:
    jul_bikemil_df.to_csv(f, encoding="utf-8", index=False, header=False)

##### --- August ---

In [ ]:
# Read file and print the number of records and data types
csv_name = "./2019/201908.csv"
aug_df = pd.read_csv(csv_name, low_memory=False) 

In [ ]:
# Add columns
aug_df.insert(0, 'year', 2019)
aug_df.insert(1, 'month', 'Aug')
aug_df.insert(3, 'tripdurmin', 0)
aug_df.insert(5, 'starthour', 0)
aug_df.insert(6, 'weekday', 0)
aug_df.insert(16, 'distance', 0)
aug_df.insert(21, 'age', 0)
aug_df.insert(22, 'sgender', '')
aug_df.insert(23, 'season', 'Summer')
aug_df.insert(24, 'mileage', 0)
print("Records Aug-2019 : " + "\n" + str(aug_df.count()))

In [ ]:
# Define values for gender
aug_df.loc[aug_df['gender'] == 0, 'sgender'] = 'Unknown'
aug_df.loc[aug_df['gender'] == 1, 'sgender'] = 'Male'
aug_df.loc[aug_df['gender'] == 2, 'sgender'] = 'Female'

# Calculate the age of the person (in 2019)
aug_df['age'] = 2019 - aug_df['birth year']

# durantion of the trip from seconds to minutes
aug_df['tripdurmin'] = aug_df['tripduration'] / 60

# Getting hour fron the 'startime'
aug_df['starthour'] = pd.to_datetime(aug_df['starttime']).dt.strftime('%H')

# Getting weekday fron the 'startime'
aug_df['weekday'] = pd.to_datetime(aug_df['starttime']).dt.strftime('%A')

In [ ]:
# Calculate the distance in miles from start station to end station
aug_df['distance'] = aug_df.apply(lambda row: distance((row['start station latitude'],row['start station longitude']), 
                         (row['end station latitude'], row['end station longitude'])),
                         axis=1)

In [ ]:
# Transform the values for mileage estimates - assumed speed of 7.456 miles per hour, up to two hours. 
aug_df.loc[aug_df['tripdurmin'] <= 120, 'mileage'] = (aug_df['tripdurmin'] / 60) * 7.456

# Trips over 2 hrs max-out at 14.9 miles
aug_df.loc[aug_df['tripdurmin'] > 120, 'mileage'] = (aug_df['tripdurmin'] / 60) * 14.9

In [ ]:
# Usertype + Gender
aug_user_df = aug_df.groupby(['usertype', 'sgender']).agg({i:'count' for i in aug_df.columns[19:20]})
aug_user_df.reset_index(inplace=True)
aug_user_df.rename(columns={"sgender":"gender", "birth year":"trips"}, inplace=True)
aug_user_df.insert(0, 'year', 2019)
aug_user_df.insert(1, 'month', 'Aug')
aug_user_df

In [ ]:
# Trips by hour, season and weekday
aug_season_df = aug_df.groupby(['starthour', 'season', 'weekday']).agg({i:'count' for i in aug_df.columns[19:20]})
aug_season_df.reset_index(inplace=True)
aug_season_df.rename(columns={"birth year":"trips"}, inplace=True)
aug_season_df.insert(0, 'year', 2019)
aug_season_df.insert(1, 'month', 'Aug')
aug_season_df

In [ ]:
# Start Stations
aug_stat_df = aug_df.groupby(['start station id', 'start station name', 'start station latitude', 'start station longitude']).agg({i:'count' for i in aug_df.columns[16:17]})
aug_stat_df.reset_index(inplace=True)
aug_stat_df.rename(columns={"start station id" : "stationid", "distance":"starttrips", "start station name": "startstatname"}, inplace=True)
aug_stat_df.insert(0, 'year', 2019)
aug_stat_df.insert(1, 'month', 'Aug')
aug_stat_df

In [ ]:
# End Stations
aug_statend_df = aug_df.groupby(['end station id', 'end station name', 'end station latitude', 'end station longitude']).agg({i:'count' for i in aug_df.columns[16:17]})
aug_statend_df.reset_index(inplace=True)
aug_statend_df.rename(columns={"end station id" : "stationid", "distance":"endtrips", "end station name": "endstatname"}, inplace=True)
aug_statend_df

In [ ]:
# Merge the start trips with the end trips by station id
aug_stat_df = aug_stat_df.merge(aug_statend_df, how="outer", left_on='stationid', right_on='stationid', suffixes=('_left', '_right'))
aug_stat_df

In [ ]:
# Ages vs Trip duration
aug_agedur_df = aug_df.groupby(by=['age','usertype'])['tripdurmin'].agg(['count', 'mean'])
aug_agedur_df.reset_index(inplace=True)
aug_agedur_df.insert(0, 'year', 2019)
aug_agedur_df.insert(1, 'month', 'Aug')
aug_agedur_df

In [ ]:
# Distance from start station to end station for each bike ID
aug_bike_df = aug_df.groupby(by=['bikeid'])['distance'].agg(['count', sum, 'mean', 'std'])
aug_bike_df.reset_index(inplace=True)
aug_bike_df.rename(columns={"count":"trips", "sum" : "totmiles", "mean" : "avgmiles", "std" : "stdmiles"}, inplace=True)
aug_bike_df.insert(0, 'year', 2019)
aug_bike_df.insert(1, 'month', 'Aug')
aug_bike_df

In [ ]:
# Calculate the total mileage for each bike ID
aug_bikemil_df = aug_df.groupby(by=['bikeid'])['mileage'].agg(['count', sum, 'mean', 'std'])
aug_bikemil_df.reset_index(inplace=True)
aug_bikemil_df.rename(columns={"count":"trips", "sum" : "totmileage", "mean" : "avgmileage", "std" : "stdmileage"}, inplace=True)
aug_bikemil_df.insert(0, 'year', 2019)
aug_bikemil_df.insert(1, 'month', 'Aug')
aug_bikemil_df

In [ ]:
# Append DataFrames to CSVs
with open('./2019/user2019.csv', 'a') as f:
    aug_user_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/season2019.csv', 'a') as f:
    aug_season_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/station2019.csv', 'a') as f:
    aug_stat_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/agedur2019.csv', 'a') as f:
    aug_agedur_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/bike2019.csv', 'a') as f:
    aug_bike_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/mileage2019.csv', 'a') as f:
    aug_bikemil_df.to_csv(f, encoding="utf-8", index=False, header=False)

##### --- September ---

In [ ]:
# Read file and print the number of records and data types
csv_name = "./2019/201909.csv"
sep_df = pd.read_csv(csv_name, low_memory=False) 

In [ ]:
# Add columns
sep_df.insert(0, 'year', 2019)
sep_df.insert(1, 'month', 'Sep')
sep_df.insert(3, 'tripdurmin', 0)
sep_df.insert(5, 'starthour', 0)
sep_df.insert(6, 'weekday', 0)
sep_df.insert(16, 'distance', 0)
sep_df.insert(21, 'age', 0)
sep_df.insert(22, 'sgender', '')
sep_df.insert(23, 'season', 'Autumn')
sep_df.insert(24, 'mileage', 0)
print("Records Sep-2019 : " + "\n" + str(sep_df.count()))

In [ ]:
# Define values for gender
sep_df.loc[sep_df['gender'] == 0, 'sgender'] = 'Unknown'
sep_df.loc[sep_df['gender'] == 1, 'sgender'] = 'Male'
sep_df.loc[sep_df['gender'] == 2, 'sgender'] = 'Female'

# Calculate the age of the person (in 2019)
sep_df['age'] = 2019 - sep_df['birth year']

# durantion of the trip from seconds to minutes
sep_df['tripdurmin'] = sep_df['tripduration'] / 60

# Getting hour fron the 'startime'
sep_df['starthour'] = pd.to_datetime(sep_df['starttime']).dt.strftime('%H')

# Getting weekday fron the 'startime'
sep_df['weekday'] = pd.to_datetime(sep_df['starttime']).dt.strftime('%A')

In [ ]:
# Calculate the distance in miles from start station to end station
sep_df['distance'] = sep_df.apply(lambda row: distance((row['start station latitude'],row['start station longitude']), 
                         (row['end station latitude'], row['end station longitude'])),
                         axis=1)

In [ ]:
# Transform the values for mileage estimates - assumed speed of 7.456 miles per hour, up to two hours. 
sep_df.loc[sep_df['tripdurmin'] <= 120, 'mileage'] = (sep_df['tripdurmin'] / 60) * 7.456

# Trips over 2 hrs max-out at 14.9 miles
sep_df.loc[sep_df['tripdurmin'] > 120, 'mileage'] = (sep_df['tripdurmin'] / 60) * 14.9

In [ ]:
# Usertype + Gender
sep_user_df = sep_df.groupby(['usertype', 'sgender']).agg({i:'count' for i in sep_df.columns[19:20]})
sep_user_df.reset_index(inplace=True)
sep_user_df.rename(columns={"sgender":"gender", "birth year":"trips"}, inplace=True)
sep_user_df.insert(0, 'year', 2019)
sep_user_df.insert(1, 'month', 'Sep')
sep_user_df

In [ ]:
# Trips by hour, season and weekday
sep_season_df = sep_df.groupby(['starthour', 'season', 'weekday']).agg({i:'count' for i in sep_df.columns[19:20]})
sep_season_df.reset_index(inplace=True)
sep_season_df.rename(columns={"birth year":"trips"}, inplace=True)
sep_season_df.insert(0, 'year', 2019)
sep_season_df.insert(1, 'month', 'Sep')
sep_season_df

In [ ]:
# Start Stations
sep_stat_df = sep_df.groupby(['start station id', 'start station name', 'start station latitude', 'start station longitude']).agg({i:'count' for i in sep_df.columns[16:17]})
sep_stat_df.reset_index(inplace=True)
sep_stat_df.rename(columns={"start station id" : "stationid", "distance":"starttrips", "start station name": "startstatname"}, inplace=True)
sep_stat_df.insert(0, 'year', 2019)
sep_stat_df.insert(1, 'month', 'Sep')
sep_stat_df

In [ ]:
# End Stations
sep_statend_df = sep_df.groupby(['end station id', 'end station name', 'end station latitude', 'end station longitude']).agg({i:'count' for i in sep_df.columns[16:17]})
sep_statend_df.reset_index(inplace=True)
sep_statend_df.rename(columns={"end station id" : "stationid", "distance":"endtrips", "end station name": "endstatname"}, inplace=True)
sep_statend_df

In [ ]:
# Merge the start trips with the end trips by station id
sep_stat_df = sep_stat_df.merge(sep_statend_df, how="outer", left_on='stationid', right_on='stationid', suffixes=('_left', '_right'))
sep_stat_df

In [ ]:
# Ages vs Trip duration
sep_agedur_df = sep_df.groupby(by=['age','usertype'])['tripdurmin'].agg(['count', 'mean'])
sep_agedur_df.reset_index(inplace=True)
sep_agedur_df.insert(0, 'year', 2019)
sep_agedur_df.insert(1, 'month', 'Sep')
sep_agedur_df

In [ ]:
# Distance from start station to end station for each bike ID
sep_bike_df = sep_df.groupby(by=['bikeid'])['distance'].agg(['count', sum, 'mean', 'std'])
sep_bike_df.reset_index(inplace=True)
sep_bike_df.rename(columns={"count":"trips", "sum" : "totmiles", "mean" : "avgmiles", "std" : "stdmiles"}, inplace=True)
sep_bike_df.insert(0, 'year', 2019)
sep_bike_df.insert(1, 'month', 'Sep')
sep_bike_df

In [ ]:
# Calculate the total mileage for each bike ID
sep_bikemil_df = sep_df.groupby(by=['bikeid'])['mileage'].agg(['count', sum, 'mean', 'std'])
sep_bikemil_df.reset_index(inplace=True)
sep_bikemil_df.rename(columns={"count":"trips", "sum" : "totmileage", "mean" : "avgmileage", "std" : "stdmileage"}, inplace=True)
sep_bikemil_df.insert(0, 'year', 2019)
sep_bikemil_df.insert(1, 'month', 'Sep')
sep_bikemil_df

In [ ]:
# Append DataFrames to CSVs
with open('./2019/user2019.csv', 'a') as f:
    sep_user_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/season2019.csv', 'a') as f:
    sep_season_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/station2019.csv', 'a') as f:
    sep_stat_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/agedur2019.csv', 'a') as f:
    sep_agedur_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/bike2019.csv', 'a') as f:
    sep_bike_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/mileage2019.csv', 'a') as f:
    sep_bikemil_df.to_csv(f, encoding="utf-8", index=False, header=False)

##### --- October ---

In [ ]:
# Read file and print the number of records and data types
csv_name = "./2019/201910.csv"
oct_df = pd.read_csv(csv_name, low_memory=False) 

In [ ]:
# Add columns
oct_df.insert(0, 'year', 2019)
oct_df.insert(1, 'month', 'Oct')
oct_df.insert(3, 'tripdurmin', 0)
oct_df.insert(5, 'starthour', 0)
oct_df.insert(6, 'weekday', 0)
oct_df.insert(16, 'distance', 0)
oct_df.insert(21, 'age', 0)
oct_df.insert(22, 'sgender', '')
oct_df.insert(23, 'season', 'Autumn')
oct_df.insert(24, 'mileage', 0)
print("Records Oct-2019 : " + "\n" + str(oct_df.count()))

In [ ]:
# Define values for gender
oct_df.loc[sep_df['gender'] == 0, 'sgender'] = 'Unknown'
oct_df.loc[sep_df['gender'] == 1, 'sgender'] = 'Male'
oct_df.loc[sep_df['gender'] == 2, 'sgender'] = 'Female'

# Calculate the age of the person (in 2019)
oct_df['age'] = 2019 - oct_df['birth year']

# durantion of the trip from seconds to minutes
oct_df['tripdurmin'] = oct_df['tripduration'] / 60

# Getting hour fron the 'startime'
oct_df['starthour'] = pd.to_datetime(oct_df['starttime']).dt.strftime('%H')

# Getting weekday fron the 'startime'
oct_df['weekday'] = pd.to_datetime(oct_df['starttime']).dt.strftime('%A')

In [ ]:
# Calculate the distance in miles from start station to end station
oct_df['distance'] = oct_df.apply(lambda row: distance((row['start station latitude'],row['start station longitude']), 
                         (row['end station latitude'], row['end station longitude'])),
                         axis=1)

In [ ]:
# Transform the values for mileage estimates - assumed speed of 7.456 miles per hour, up to two hours. 
oct_df.loc[oct_df['tripdurmin'] <= 120, 'mileage'] = (oct_df['tripdurmin'] / 60) * 7.456

# Trips over 2 hrs max-out at 14.9 miles
oct_df.loc[oct_df['tripdurmin'] > 120, 'mileage'] = (oct_df['tripdurmin'] / 60) * 14.9

In [ ]:
# Usertype + Gender
oct_user_df = oct_df.groupby(['usertype', 'sgender']).agg({i:'count' for i in oct_df.columns[19:20]})
oct_user_df.reset_index(inplace=True)
oct_user_df.rename(columns={"sgender":"gender", "birth year":"trips"}, inplace=True)
oct_user_df.insert(0, 'year', 2019)
oct_user_df.insert(1, 'month', 'Oct')
oct_user_df

In [ ]:
# Trips by hour, season and weekday
oct_season_df = oct_df.groupby(['starthour', 'season', 'weekday']).agg({i:'count' for i in oct_df.columns[19:20]})
oct_season_df.reset_index(inplace=True)
oct_season_df.rename(columns={"birth year":"trips"}, inplace=True)
oct_season_df.insert(0, 'year', 2019)
oct_season_df.insert(1, 'month', 'Oct')
oct_season_df

In [ ]:
# Start Stations
oct_stat_df = oct_df.groupby(['start station id', 'start station name', 'start station latitude', 'start station longitude']).agg({i:'count' for i in oct_df.columns[16:17]})
oct_stat_df.reset_index(inplace=True)
oct_stat_df.rename(columns={"start station id" : "stationid", "distance":"starttrips", "start station name": "startstatname"}, inplace=True)
oct_stat_df.insert(0, 'year', 2019)
oct_stat_df.insert(1, 'month', 'Oct')
oct_stat_df

In [ ]:
# End Stations
oct_statend_df = oct_df.groupby(['end station id', 'end station name', 'end station latitude', 'end station longitude']).agg({i:'count' for i in oct_df.columns[16:17]})
oct_statend_df.reset_index(inplace=True)
oct_statend_df.rename(columns={"end station id" : "stationid", "distance":"endtrips", "end station name": "endstatname"}, inplace=True)
oct_statend_df

In [ ]:
# Merge the start trips with the end trips by station id
oct_stat_df = oct_stat_df.merge(oct_statend_df, how="outer", left_on='stationid', right_on='stationid', suffixes=('_left', '_right'))
oct_stat_df

In [ ]:
# Ages vs Trip duration
oct_agedur_df = oct_df.groupby(by=['age','usertype'])['tripdurmin'].agg(['count', 'mean'])
oct_agedur_df.reset_index(inplace=True)
oct_agedur_df.insert(0, 'year', 2019)
oct_agedur_df.insert(1, 'month', 'Oct')
oct_agedur_df

In [ ]:
# Distance from start station to end station for each bike ID
oct_bike_df = oct_df.groupby(by=['bikeid'])['distance'].agg(['count', sum, 'mean', 'std'])
oct_bike_df.reset_index(inplace=True)
oct_bike_df.rename(columns={"count":"trips", "sum" : "totmiles", "mean" : "avgmiles", "std" : "stdmiles"}, inplace=True)
oct_bike_df.insert(0, 'year', 2019)
oct_bike_df.insert(1, 'month', 'Oct')
oct_bike_df

In [ ]:
# Calculate the total mileage for each bike ID
oct_bikemil_df = oct_df.groupby(by=['bikeid'])['mileage'].agg(['count', sum, 'mean', 'std'])
oct_bikemil_df.reset_index(inplace=True)
oct_bikemil_df.rename(columns={"count":"trips", "sum" : "totmileage", "mean" : "avgmileage", "std" : "stdmileage"}, inplace=True)
oct_bikemil_df.insert(0, 'year', 2019)
oct_bikemil_df.insert(1, 'month', 'Oct')
oct_bikemil_df

In [ ]:
# Append DataFrames to CSVs
with open('./2019/user2019.csv', 'a') as f:
    oct_user_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/season2019.csv', 'a') as f:
    oct_season_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/station2019.csv', 'a') as f:
    oct_stat_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/agedur2019.csv', 'a') as f:
    oct_agedur_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/bike2019.csv', 'a') as f:
    oct_bike_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/mileage2019.csv', 'a') as f:
    oct_bikemil_df.to_csv(f, encoding="utf-8", index=False, header=False)

##### --- November ---

In [ ]:
# Read file and print the number of records and data types
csv_name = "./2019/201911.csv"
nov_df = pd.read_csv(csv_name, low_memory=False) 

In [ ]:
# Add columns
nov_df.insert(0, 'year', 2019)
nov_df.insert(1, 'month', 'Nov')
nov_df.insert(3, 'tripdurmin', 0)
nov_df.insert(5, 'starthour', 0)
nov_df.insert(6, 'weekday', 0)
nov_df.insert(16, 'distance', 0)
nov_df.insert(21, 'age', 0)
nov_df.insert(22, 'sgender', '')
nov_df.insert(23, 'season', 'Autumn')
nov_df.insert(24, 'mileage', 0)
print("Records Nov-2019 : " + "\n" + str(nov_df.count()))

In [ ]:
# Define values for gender
nov_df.loc[nov_df['gender'] == 0, 'sgender'] = 'Unknown'
nov_df.loc[nov_df['gender'] == 1, 'sgender'] = 'Male'
nov_df.loc[nov_df['gender'] == 2, 'sgender'] = 'Female'

# Calculate the age of the person (in 2019)
nov_df['age'] = 2019 - nov_df['birth year']

# durantion of the trip from seconds to minutes
nov_df['tripdurmin'] = nov_df['tripduration'] / 60

# Getting hour fron the 'startime'
nov_df['starthour'] = pd.to_datetime(nov_df['starttime']).dt.strftime('%H')

# Getting weekday fron the 'startime'
nov_df['weekday'] = pd.to_datetime(nov_df['starttime']).dt.strftime('%A')

In [ ]:
# Calculate the distance in miles from start station to end station
nov_df['distance'] = nov_df.apply(lambda row: distance((row['start station latitude'],row['start station longitude']), 
                         (row['end station latitude'], row['end station longitude'])),
                         axis=1)

In [ ]:
# Transform the values for mileage estimates - assumed speed of 7.456 miles per hour, up to two hours. 
nov_df.loc[nov_df['tripdurmin'] <= 120, 'mileage'] = (nov_df['tripdurmin'] / 60) * 7.456

# Trips over 2 hrs max-out at 14.9 miles
nov_df.loc[nov_df['tripdurmin'] > 120, 'mileage'] = (nov_df['tripdurmin'] / 60) * 14.9

In [ ]:
# Usertype + Gender
nov_user_df = nov_df.groupby(['usertype', 'sgender']).agg({i:'count' for i in nov_df.columns[19:20]})
nov_user_df.reset_index(inplace=True)
nov_user_df.rename(columns={"sgender":"gender", "birth year":"trips"}, inplace=True)
nov_user_df.insert(0, 'year', 2019)
nov_user_df.insert(1, 'month', 'Nov')
nov_user_df

In [ ]:
# Trips by hour, season and weekday
nov_season_df = nov_df.groupby(['starthour', 'season', 'weekday']).agg({i:'count' for i in nov_df.columns[19:20]})
nov_season_df.reset_index(inplace=True)
nov_season_df.rename(columns={"birth year":"trips"}, inplace=True)
nov_season_df.insert(0, 'year', 2019)
nov_season_df.insert(1, 'month', 'Nov')
nov_season_df

In [ ]:
# Start Stations
nov_stat_df = nov_df.groupby(['start station id', 'start station name', 'start station latitude', 'start station longitude']).agg({i:'count' for i in nov_df.columns[16:17]})
nov_stat_df.reset_index(inplace=True)
nov_stat_df.rename(columns={"start station id" : "stationid", "distance":"starttrips", "start station name": "startstatname"}, inplace=True)
nov_stat_df.insert(0, 'year', 2019)
nov_stat_df.insert(1, 'month', 'Nov')
nov_stat_df

In [ ]:
# End Stations
nov_statend_df = nov_df.groupby(['end station id', 'end station name', 'end station latitude', 'end station longitude']).agg({i:'count' for i in nov_df.columns[16:17]})
nov_statend_df.reset_index(inplace=True)
nov_statend_df.rename(columns={"end station id" : "stationid", "distance":"endtrips", "end station name": "endstatname"}, inplace=True)
nov_statend_df

In [ ]:
# Merge the start trips with the end trips by station id
nov_stat_df = nov_stat_df.merge(nov_statend_df, how="outer", left_on='stationid', right_on='stationid', suffixes=('_left', '_right'))
nov_stat_df

In [ ]:
# Ages vs Trip duration
nov_agedur_df = nov_df.groupby(by=['age','usertype'])['tripdurmin'].agg(['count', 'mean'])
nov_agedur_df.reset_index(inplace=True)
nov_agedur_df.insert(0, 'year', 2019)
nov_agedur_df.insert(1, 'month', 'Nov')
nov_agedur_df

In [ ]:
# Distance from start station to end station for each bike ID
nov_bike_df = nov_df.groupby(by=['bikeid'])['distance'].agg(['count', sum, 'mean', 'std'])
nov_bike_df.reset_index(inplace=True)
nov_bike_df.rename(columns={"count":"trips", "sum" : "totmiles", "mean" : "avgmiles", "std" : "stdmiles"}, inplace=True)
nov_bike_df.insert(0, 'year', 2019)
nov_bike_df.insert(1, 'month', 'Nov')
nov_bike_df

In [ ]:
# Calculate the total mileage for each bike ID
nov_bikemil_df = nov_df.groupby(by=['bikeid'])['mileage'].agg(['count', sum, 'mean', 'std'])
nov_bikemil_df.reset_index(inplace=True)
nov_bikemil_df.rename(columns={"count":"trips", "sum" : "totmileage", "mean" : "avgmileage", "std" : "stdmileage"}, inplace=True)
nov_bikemil_df.insert(0, 'year', 2019)
nov_bikemil_df.insert(1, 'month', 'Nov')
nov_bikemil_df

In [ ]:
# Append DataFrames to CSVs
with open('./2019/user2019.csv', 'a') as f:
    nov_user_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/season2019.csv', 'a') as f:
    nov_season_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/station2019.csv', 'a') as f:
    nov_stat_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/agedur2019.csv', 'a') as f:
    nov_agedur_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/bike2019.csv', 'a') as f:
    nov_bike_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/mileage2019.csv', 'a') as f:
    nov_bikemil_df.to_csv(f, encoding="utf-8", index=False, header=False)

##### --- December ---

In [ ]:
# Read file and print the number of records and data types
csv_name = "./2019/201912.csv"
dec_df = pd.read_csv(csv_name, low_memory=False) 

In [ ]:
# Add columns
dec_df.insert(0, 'year', 2019)
dec_df.insert(1, 'month', 'Dec')
dec_df.insert(3, 'tripdurmin', 0)
dec_df.insert(5, 'starthour', 0)
dec_df.insert(6, 'weekday', 0)
dec_df.insert(16, 'distance', 0)
dec_df.insert(21, 'age', 0)
dec_df.insert(22, 'sgender', '')
dec_df.insert(23, 'season', 'Winter')
dec_df.insert(24, 'mileage', 0)
print("Records Dec-2019 : " + "\n" + str(nov_df.count()))

In [ ]:
# Define values for gender
dec_df.loc[dec_df['gender'] == 0, 'sgender'] = 'Unknown'
dec_df.loc[dec_df['gender'] == 1, 'sgender'] = 'Male'
dec_df.loc[dec_df['gender'] == 2, 'sgender'] = 'Female'

# Calculate the age of the person (in 2019)
dec_df['age'] = 2019 - dec_df['birth year']

# durantion of the trip from seconds to minutes
dec_df['tripdurmin'] = dec_df['tripduration'] / 60

# Getting hour fron the 'startime'
dec_df['starthour'] = pd.to_datetime(dec_df['starttime']).dt.strftime('%H')

# Getting weekday fron the 'startime'
dec_df['weekday'] = pd.to_datetime(dec_df['starttime']).dt.strftime('%A')

In [ ]:
# Calculate the distance in miles from start station to end station
dec_df['distance'] = dec_df.apply(lambda row: distance((row['start station latitude'],row['start station longitude']), 
                         (row['end station latitude'], row['end station longitude'])),
                         axis=1)

In [ ]:
# Transform the values for mileage estimates - assumed speed of 7.456 miles per hour, up to two hours. 
dec_df.loc[dec_df['tripdurmin'] <= 120, 'mileage'] = (dec_df['tripdurmin'] / 60) * 7.456

# Trips over 2 hrs max-out at 14.9 miles
dec_df.loc[dec_df['tripdurmin'] > 120, 'mileage'] = (dec_df['tripdurmin'] / 60) * 14.9

In [ ]:
# Usertype + Gender
dec_user_df = dec_df.groupby(['usertype', 'sgender']).agg({i:'count' for i in dec_df.columns[19:20]})
dec_user_df.reset_index(inplace=True)
dec_user_df.rename(columns={"sgender":"gender", "birth year":"trips"}, inplace=True)
dec_user_df.insert(0, 'year', 2019)
dec_user_df.insert(1, 'month', 'Dec')
dec_user_df

In [ ]:
# Trips by hour, season and weekday
dec_season_df = dec_df.groupby(['starthour', 'season', 'weekday']).agg({i:'count' for i in dec_df.columns[19:20]})
dec_season_df.reset_index(inplace=True)
dec_season_df.rename(columns={"birth year":"trips"}, inplace=True)
dec_season_df.insert(0, 'year', 2019)
dec_season_df.insert(1, 'month', 'Dec')
dec_season_df

In [ ]:
# Start Stations
dec_stat_df = dec_df.groupby(['start station id', 'start station name', 'start station latitude', 'start station longitude']).agg({i:'count' for i in dec_df.columns[16:17]})
dec_stat_df.reset_index(inplace=True)
dec_stat_df.rename(columns={"start station id" : "stationid", "distance":"starttrips", "start station name": "startstatname"}, inplace=True)
dec_stat_df.insert(0, 'year', 2019)
dec_stat_df.insert(1, 'month', 'Dec')
dec_stat_df

In [ ]:
# End Stations
dec_statend_df = dec_df.groupby(['end station id', 'end station name', 'end station latitude', 'end station longitude']).agg({i:'count' for i in dec_df.columns[16:17]})
dec_statend_df.reset_index(inplace=True)
dec_statend_df.rename(columns={"end station id" : "stationid", "distance":"endtrips", "end station name": "endstatname"}, inplace=True)
dec_statend_df

In [ ]:
# Merge the start trips with the end trips by station id
dec_stat_df = dec_stat_df.merge(dec_statend_df, how="outer", left_on='stationid', right_on='stationid', suffixes=('_left', '_right'))
dec_stat_df

In [ ]:
# Ages vs Trip duration
dec_agedur_df = dec_df.groupby(by=['age','usertype'])['tripdurmin'].agg(['count', 'mean'])
dec_agedur_df.reset_index(inplace=True)
dec_agedur_df.insert(0, 'year', 2019)
dec_agedur_df.insert(1, 'month', 'Dec')
dec_agedur_df

In [ ]:
# Distance from start station to end station for each bike ID
dec_bike_df = dec_df.groupby(by=['bikeid'])['distance'].agg(['count', sum, 'mean', 'std'])
dec_bike_df.reset_index(inplace=True)
dec_bike_df.rename(columns={"count":"trips", "sum" : "totmiles", "mean" : "avgmiles", "std" : "stdmiles"}, inplace=True)
dec_bike_df.insert(0, 'year', 2019)
dec_bike_df.insert(1, 'month', 'Dec')
dec_bike_df

In [ ]:
# Calculate the total mileage for each bike ID
dec_bikemil_df = dec_df.groupby(by=['bikeid'])['mileage'].agg(['count', sum, 'mean', 'std'])
dec_bikemil_df.reset_index(inplace=True)
dec_bikemil_df.rename(columns={"count":"trips", "sum" : "totmileage", "mean" : "avgmileage", "std" : "stdmileage"}, inplace=True)
dec_bikemil_df.insert(0, 'year', 2019)
dec_bikemil_df.insert(1, 'month', 'Dec')
dec_bikemil_df

In [ ]:
# Append DataFrames to CSVs
with open('./2019/user2019.csv', 'a') as f:
    dec_user_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/season2019.csv', 'a') as f:
    dec_season_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/station2019.csv', 'a') as f:
    dec_stat_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/agedur2019.csv', 'a') as f:
    dec_agedur_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/bike2019.csv', 'a') as f:
    dec_bike_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/mileage2019.csv', 'a') as f:
    dec_bikemil_df.to_csv(f, encoding="utf-8", index=False, header=False)

In [ ]:
# Append DataFrames to CSVs
with open('./2019/user2019.csv', 'a') as f:
    nov_user_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/season2019.csv', 'a') as f:
    nov_season_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/station2019.csv', 'a') as f:
    nov_stat_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/agedur2019.csv', 'a') as f:
    nov_agedur_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/bike2019.csv', 'a') as f:
    nov_bike_df.to_csv(f, encoding="utf-8", index=False, header=False)

with open('./2019/mileage2019.csv', 'a') as f:
    nov_bikemil_df.to_csv(f, encoding="utf-8", index=False, header=False)